# Chicago Car Crashes Analysis

Introduction text to come

In [1]:
from datetime import datetime
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_validate
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import make_pipeline


## Import Data

Three distinct datasets are used in this analysis:
- Crashes
- People
- Socially Disadvantaged Districts

These datasets are large.  Specifying the specific columns to import for each dataset will limit the time it takes to bring them into dataframes, and will minimize system resources

In [2]:
crashes_usecols = ['LATITUDE','LONGITUDE','CRASH_RECORD_ID','CRASH_DATE','DEVICE_CONDITION',\
                   'WEATHER_CONDITION','LIGHTING_CONDITION','ALIGNMENT','ROADWAY_SURFACE_COND',\
                   'INJURIES_FATAL','INJURIES_INCAPACITATING','CRASH_HOUR','CRASH_DAY_OF_WEEK']

people_usecols = ['CRASH_RECORD_ID','SAFETY_EQUIPMENT','PHYSICAL_CONDITION']

In [3]:
# Import CSV files into DataFrames
crashes_df = pd.read_csv('Data/Traffic_Crashes_-_Crashes_20240412.csv', usecols=crashes_usecols)
people_df = pd.read_csv('Data/Traffic_Crashes_-_People_20240412.csv', usecols=people_usecols)

In [4]:
# Load the SHP file into a GeoDataFrame
districts_gdf = gpd.read_file('Data\SD_geo.shp')

## Preprocessing

Preprocessing discussion - for later

#### Crashes dataset
- Create flag for records within disadvantaged districts
- Create bins for: (1) roadway conditions, (2) roadway alignment, (3) roadway devices, (4) weather, (5) lighting, and (6) surface conditions

#### Geographic analysis
- Drop records without locations
- Convert the Crashes DataFrame into a GeoDataFrame so that the coordinates can be turned into Points using the Shapely package
- Ensure that the resulting GDF uses the same reference system as the imported districts GDF
- Spatially join the two GDFs, checking to see if each point is within one of the districts and assigning 1's to those
- Drop unnecessary columns and convert back to a DataFrame, which is computationally more efficient

In [5]:
# Drop values with no location information
crashes_df.dropna(subset=['LATITUDE','LONGITUDE'], inplace=True)

# Convert DataFrame into GeoDataFrame
crashes_df['geometry'] = crashes_df.apply(lambda row: Point(row['LONGITUDE'], row['LATITUDE']), axis=1)
crashes_gdf = gpd.GeoDataFrame(crashes_df, geometry='geometry')

# Ensure that both GeoDataFrames use the same CRS
crashes_gdf.crs = districts_gdf.crs

# Spatial join the GeoDataFrames
joined_gdf = gpd.sjoin(crashes_gdf, districts_gdf, how='left', predicate='within')

# Add flag for crashes that are within a district
joined_gdf['WITHIN_DISTRICT'] = joined_gdf['index_right'].apply(lambda x: 1 if pd.notnull(x) else 0)

# Drop the geometry, index_right, LATITUDE and LONGITUDE columns
joined_gdf.drop(columns=['LATITUDE','LONGITUDE','geometry','index_right'], axis=1, inplace=True)

# Convert back to DataFrame
crashes_flag_df = pd.DataFrame(joined_gdf)

#### Prepare datetime data for analysis
- Create a helper function to convert each date to a season
- Convert each date string to a season
- Group days of the week into weekday/weekend categories
- Group hours of the day into thematic blocks
- Drop unnecessary columns

In [6]:
# Helper function to convert dates to seasons

def get_season(date):
    year = date.year
    seasons = [('winter', datetime(year, 1, 1).date(), datetime(year, 2, 28).date()),
               ('spring', datetime(year, 3, 1).date(), datetime(year, 5, 31).date()),
               ('summer', datetime(year, 6, 1).date(), datetime(year, 8, 31).date()),
               ('autumn', datetime(year, 9, 1).date(), datetime(year, 11, 30).date()),
               ('winter', datetime(year, 12, 1).date(), datetime(year, 12, 31).date())]
    if date.year % 4 == 0:  # leap year check
        seasons[0] = ('winter', datetime(year, 1, 1).date(), datetime(year, 2, 29).date())
    
    for season, start, end in seasons:
        if start <= date <= end:
            return season

    return 'Date is out of range'

In [7]:
# Convert date field to season
crashes_df['SEASON'] = crashes_df['CRASH_DATE'].apply(lambda x: 
                                                      get_season(datetime.strptime(x[:10],'%m/%d/%Y').date()))

# Group CRASH_DAY_OF_WEEK into weekdays and weekends
weekday_mask = [2,3,4,5,6]
crashes_flag_df['CRASH_WEEKEND'] = crashes_flag_df['CRASH_DAY_OF_WEEK'].apply(lambda x: 0 if x in weekday_mask else 1)

# Group hours into time blocks (extra bin at end to ensure correct bin treatment)
bins = [-1, 6, 9, 15, 19, 23, 24]
labels = ['NIGHT', 'MORNING_RUSH', 'MIDDAY', 'EVENING_RUSH', 'NIGHT', 'NIGHT']
crashes_flag_df['TIME_BLOCK'] = pd.cut(crashes_flag_df['CRASH_HOUR'], bins=bins, labels=labels, right=True, ordered=False)

# Drop unnecessary columns
crashes_df.drop(columns=['CRASH_DATE','CRASH_DAY_OF_WEEK','CRASH_HOUR'], axis=1, inplace=True)                                                      

#### Category flags
- Convert string categories into 1/0 where 1's signify potential dangerous conditions
- Categories include malfunctioning road device, bad weather conditions, poor lighting, non-straight and level roads and unsafe surfaces
- Create a target column equal to 1 if there are any fatal or incapacitating injuries
- Drop unnecessary columns

In [8]:
device_mask = ['NO CONTROLS','FUNCTIONING PROPERLY']
weather_mask = ['CLEAR','UNKNOWN']
lighting_mask = ['DAYLIGHT']
alignment_mask = ['STRAIGHT AND LEVEL']
surface_mask = ['DRY','UNKNOWN']

crashes_flag_df['DEVICE_FLAG'] = crashes_flag_df['DEVICE_CONDITION'].apply(lambda x: 0 if x in device_mask else 1)
crashes_flag_df['WEATHER_FLAG'] = crashes_flag_df['WEATHER_CONDITION'].apply(lambda x: 0 if x in weather_mask else 1)
crashes_flag_df['LIGHTING_FLAG'] = crashes_flag_df['LIGHTING_CONDITION'].apply(lambda x: 0 if x in lighting_mask else 1)
crashes_flag_df['ALIGNMENT_FLAG'] = crashes_flag_df['ALIGNMENT'].apply(lambda x: 0 if x in alignment_mask else 1)
crashes_flag_df['SURFACE_FLAG'] = crashes_flag_df['ROADWAY_SURFACE_COND'].apply(lambda x: 0 if x in surface_mask else 1)

# Flag for serious accidents (fatal + incapacitating), which is the target
crashes_flag_df['TARGET'] = crashes_flag_df.apply(lambda row: 1 if 
                                                  (row['INJURIES_FATAL']+row['INJURIES_INCAPACITATING'] > 0) else 0,
                                                  axis=1)

# Drop unnecessary columns
crashes_flag_df = crashes_flag_df.drop(columns=['DEVICE_CONDITION', 'WEATHER_CONDITION', 'LIGHTING_CONDITION',\
                                                'ALIGNMENT', 'ROADWAY_SURFACE_COND',\
                                               'INJURIES_FATAL', 'INJURIES_INCAPACITATING'], axis=1)

#### People Dataset
This dataset includes all people involved with any crash (e.g. the driver of each car, each passenger), so there is more than one line per crash.  Since this analysis predicts the effect of crashes, all relevant information must be extracted and processed into per-crash form.  This is accomplished by converting each feature into a single flag per crash.
- Flag vehicle operators with compromised physical features (e.g. alcohol, drugs, tired)
- Flag participants that failed to use vehicle safety equipment properly
- Group people by CRASH_ID and apply flag if applicable
- Drop unnecessary columns

In [9]:
# Masks to assist in binning the PHYSICAL_CONDITION and SAFETY_EQUIPMENT fields
PhysicalMask = ['NORMAL', 'UNKNOWN']
SafetyMask = ['SAFETY BELT USED', 'USAGE UNKNOWN', 'CHILD RESTRAINT USED', 'CHILD RESTRAINT - FORWARD FACING'\
             'BICYCLE HELMET (PEDACYCLIST INVOLVED ONLY)', 'CHILD RESTRAINT - TYPE UNKNOWN',\
             'CHILD RESTRAINT - REAR FACING', 'HELMET USED', 'DOT COMPLIANT MOTORCYCLE HELMET',\
             'BOOSTER SEAT', 'WHEELCHAIR', 'STRETCHER']

# Bin all problematic physical and safety conditions and tag with a 1 
people_df['PHYSICAL_FLAG'] = people_df['PHYSICAL_CONDITION'].apply(lambda x: 0 if x in PhysicalMask else 1)
people_df['SAFETY_FLAG'] = people_df['SAFETY_EQUIPMENT'].apply(lambda x: 0 if x in SafetyMask else 1)

# Drop the original columns
people_df = people_df.drop(columns=['PHYSICAL_CONDITION', 'SAFETY_EQUIPMENT'], axis=1)

# For each crash, tag if at least one element had a safety or physical problem
safety_flag = people_df.groupby('CRASH_RECORD_ID')['SAFETY_FLAG'].max().reset_index()
safety_flag.rename(columns={'SAFETY_FLAG': 'SAFETY_PROBLEM'}, inplace=True)
people_df = people_df.drop('SAFETY_FLAG', axis=1).merge(safety_flag, on='CRASH_RECORD_ID', how='left')

physical_flag = people_df.groupby('CRASH_RECORD_ID')['PHYSICAL_FLAG'].max().reset_index()
physical_flag.rename(columns={'PHYSICAL_FLAG': 'PHYSICAL_PROBLEM'}, inplace=True)
people_df = people_df.drop('PHYSICAL_FLAG', axis=1).merge(physical_flag, on='CRASH_RECORD_ID', how='left')

# Drop remaining duplicate rows
people_df.drop_duplicates(inplace=True)

In [14]:
# Merge the dataframes
combined_df = crashes_flag_df.merge(people_df, on='CRASH_RECORD_ID', how='inner')

In [15]:
combined_df.TARGET.sum()

14723

## Modeling

## Conclusion

For the future:
- Proximity to holidays